In [51]:
import pandas as pd
import numpy as np
import random as r

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [52]:
file_path = 'Prado_pp_df_control.pkl'
df_original = pd.read_pickle(file_path)
df = df_original.copy()
columns_to_drop = ['lead_time_mismatch', 'assignment_weekday']
df = df.drop(columns=columns_to_drop)
df.head()

,month,appointment_weekday,desired_weekday,gender,age,population_group,regime,specialties,type_of_consultation,specialty_classification,appointment_time,desired_lead_time,real_lead_time,missed_appointment
77841,may,friday,thursday,female,71,general population,contributory,sports medicine,normal,specialized,0.510417,0,1,1
77842,may,tuesday,monday,female,71,general population,contributory,sports medicine,normal,specialized,0.534722,0,1,1
77843,may,thursday,wednesday,female,71,general population,contributory,sports medicine,normal,specialized,0.527778,0,1,1
77844,may,friday,friday,female,71,general population,contributory,sports medicine,normal,specialized,0.586806,0,0,1
77845,may,tuesday,tuesday,female,72,general population,contributory,sports medicine,normal,specialized,0.541667,0,0,1


In [53]:
df = pd.get_dummies(df, drop_first=True)

df.columns = df.columns.str.replace(' ', '_')

X = df.drop(columns=['missed_appointment'])
print(X.columns)
y = df['missed_appointment']

Index(['age', 'appointment_time', 'desired_lead_time', 'real_lead_time',
       'month_august', 'month_december', 'month_february', 'month_january',
       'month_july', 'month_june', 'month_march', 'month_may',
       'month_november', 'month_october', 'month_september',
       'appointment_weekday_monday', 'appointment_weekday_saturday',
       'appointment_weekday_thursday', 'appointment_weekday_tuesday',
       'appointment_weekday_wednesday', 'desired_weekday_monday',
       'desired_weekday_saturday', 'desired_weekday_sunday',
       'desired_weekday_thursday', 'desired_weekday_tuesday',
       'desired_weekday_wednesday', 'gender_male',
       'population_group_general_population',
       'population_group_other_ethnicities',
       'population_group_reintegrated_or_demobilized',
       'population_group_street_dweller',
       'population_group_victims_of_armed_conflict', 'regime_not_applicable',
       'regime_special', 'regime_subsidized',
       'specialties_bioenergetic_med

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
rf = RandomForestClassifier(random_state=42)

rf.fit(X_train, y_train)
y_pred_proba = rf.predict_proba(X_test)
y_pred_proba[:,-1]
y_pred_proba_1 = y_pred_proba[:,-1]
roc_auc_score(y_test, y_pred_proba_1)

0.7307189459609932

In [56]:
random_patient = X_test.sample(1)
random_patient
# inference on that random patient
rf.predict_proba(random_patient)[:,-1][0]

0.04

In [57]:
# patient class with the attributes as the columns of the df
class Patient:
    def __init__(self, id, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
        self.id = id
        self.proba = 0
        self.protected = False
        self.attendance = False
        self.assigned = False
        self.day_of_call = 0
        self.num_slot=-1
        self.overbooked = False

        if self.regime_subsidized == 1:
            self.protected = True

    def predict_proba(self, model):

        overbooking_proba_protected = 0.5
        overbooking_proba_non_protected = 0.5

        # prediccion sobre los atributos del paciente excepto id, predicted_proba y protected
        patient_data = pd.DataFrame([self.__dict__])
        patient_data = patient_data.drop(columns=['id', 'proba', 'protected', 'attendance', 'assigned','day_of_call','num_slot', 'overbooked'])

        # positive prediction
        prediction = model.predict_proba(patient_data)[:, -1][0]

        self.proba = prediction

        # define if patient has to be overbooked given its class
        if self.protected:
            if self.proba > overbooking_proba_protected:
                self.overbooked = True
        else:
            if self.proba > overbooking_proba_non_protected:
                self.overbooked = True

        # print(f"Patient {self.id} predicted probability: {prediction}")
        return prediction

    def properties(self):
        print(f"Patient {self.id} properties:")
        print(self.__dict__)

In [58]:
# converting all the patients to a list of patient objects
patients = [Patient(i, **row) for i, row in enumerate(X.to_dict(orient='records'))]

In [59]:
df.head()

,age,appointment_time,desired_lead_time,real_lead_time,missed_appointment,month_august,month_december,month_february,month_january,month_july,...,specialties_sports_medicine,specialties_toxicology,specialties_urology,specialties_vascular_surgery,type_of_consultation_post-surgical,type_of_consultation_pre-surgical,type_of_consultation_procedure,specialty_classification_not_applicable,specialty_classification_specialized,specialty_classification_subspecialized
77841,71,0.510417,0,1,1,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
77842,71,0.534722,0,1,1,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
77843,71,0.527778,0,1,1,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
77844,71,0.586806,0,0,1,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
77845,72,0.541667,0,0,1,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False


In [60]:
patients[4].appointment_time

0.541666667

In [61]:
patient = patients[10]
# patient.properties()

print(f"Patient {patient.id} prediction before method: {patient.proba}")
patient.predict_proba(rf)
print(f"Patient {patient.id} prediction after method: {patient.proba}") 

Patient 10 prediction before method: 0
Patient 10 prediction after method: 0.98


In [62]:
patient = patients[8000]
# patient.properties()

print(f"Patient {patient.id} prediction before method: {patient.proba}")
patient.predict_proba(rf)
print(f"Patient {patient.id} prediction after method: {patient.proba}") 

Patient 8000 prediction before method: 0
Patient 8000 prediction after method: 0.01


In [63]:
patient = patients[10000]

# without value because the initial value is 0
print(f"Patient {patient.id} prediction before method: {patient.proba}")
patient.predict_proba(rf)
print(f"Patient {patient.id} prediction after predict method: {patient.proba}") 

patient.real_lead_time = 1000
patient.predict_proba(rf)

print(f"Patient {patient.id} prediction after changing lead time: {patient.proba}") 

Patient 10000 prediction before method: 0
Patient 10000 prediction after predict method: 0.24
Patient 10000 prediction after changing lead time: 0.34


In [64]:
i = 100
print(f"Patient {patients[i].id} - Protected: {patients[i].protected} (regime_subsidized {patients[i].regime_subsidized})")

i = 1725
print(f"Patient {patients[i].id} - Protected: {patients[i].protected} (regime_subsidized {patients[i].regime_subsidized})")

Patient 100 - Protected: False (regime_subsidized False)
Patient 1725 - Protected: True (regime_subsidized True)


In [65]:
def random_patient_sample(patients, sample_size ):
    
    filtered_patients = patients

    # inicialmente no vamos a filtrar por especialidad
    # # Filter patients by specialty (general medicine)
    # filtered_patients = [patient for patient in patients if patient.specialties_general_medicine == 1]
    
    # Randomly sample from filtered patients
    sampled_patients = [filtered_patients[i] for i in np.random.choice(len(filtered_patients), sample_size, replace=False)]
    
    return sampled_patients

In [66]:
import numpy as np

def random_patient_sample(patients, sample_size):
    # Separate patients into two groups based on the 'protected' characteristic
    protected_true = [patient for patient in patients if patient.protected]
    protected_false = [patient for patient in patients if not patient.protected]

    # Calculate sample sizes for each group - Aca definimos la proporcion de la clase protegida
    total_sample_size = sample_size
    sample_size_protected_true = int(total_sample_size * 0.4)
    sample_size_protected_false = total_sample_size - sample_size_protected_true

    # Randomly select samples from each group
    sample_protected_true = [protected_true[i] for i in np.random.choice(len(protected_true), sample_size_protected_true, replace=False)]
    sample_protected_false = [protected_false[i] for i in np.random.choice(len(protected_false), sample_size_protected_false, replace=False)]

    # Concatenate the samples to get the final stratified sample
    stratified_sample = sample_protected_true + sample_protected_false

    return stratified_sample


In [67]:
def asignar_dia(patient_list_sample, num_days):

    # determinar cuantos pacientes por dia
    patients_per_day = len(patient_list_sample) // num_days
 
    # asignar dia de llamada a cada paciente
    for i, patient in enumerate(patient_list_sample):
        patient.day_of_call = i//patients_per_day
        patient.id=i
        #patient.properties()

    # organizar por dia de llamada
    patient_list_sample.sort(key=lambda x:x.day_of_call)

    return patient_list_sample

In [68]:
# Revisa los tiempos de ejecuciones al predecir

# import time
# patient = patients[0]

# start = time.time()
# for i in range(100):    
#     patient.age = np.random.randint(0, 100)
#     patient.appointment_time = np.random.rand()
#     patient.desired_lead_time = np.random.randint(0, 31)
#     patient.real_lead_time = np.random.randint(0, 31)
#     patient.predict_proba(rf)
#     if i % 1000 == 0:
#         print(f"T (change and predict {i} patients) = {(time.time() - start):.2f}s | predicted_proba: {patient.proba:.2f}")

# Clinic Class and simulation

In [69]:
class Clinic:
    def __init__(self, patients_data, appointments, simulation_days, num_serves, scheduling_days, num_hours_byday, slot_time, overbooking):
        # This is a list of patients to be served by the clinic
        self.simulation_days = simulation_days
        self.patients_list = patients_data
        self.num_serves = num_serves
        self.scheduling_days = scheduling_days
        self.num_hours_byday = num_hours_byday
        self.slot_time = slot_time # Minutes
        self.num_slots_byday_server = int(num_hours_byday * (60/slot_time)*overbooking)
        self.service_time = np.zeros(num_serves)
        self.over_time = np.zeros(num_serves)
        self.no_shows=0
        self.refused_patients=0 # When is too late
        self.total_time=0
        self.idle_time_server =np.zeros(num_serves) 
        patients_slot=1
        #Inialie the appointment with 4D array FULL OF 0
        self.appointments=appointments
        
        # inicializando este porque me daba error
        self.cwt_protected = 0
        self.cwt_non_protected = 0

        #Calcula el CWL, Desde el dia siguiente a ser solicitada la cita hasta el momento de la cita 
    def compute_waiting_time(self, day, slot, id):
        direct_time = 0
        total_time = 0
        direct_time = (slot-self.patients_list[id].num_slot)*self.slot_time
        hora_inicio = 6
        total_time = int(day-self.patients_list[id].day_of_call+1)
        total_time = total_time*60*24
        minutes = (self.slot_time*(slot))+(60*hora_inicio)
        total_time = total_time+minutes
        # Se puede retonrar tambien el tiempo total de espera PREGUNTAR COMO LO CONSIDERAM
        return direct_time

        # Convert slot_attend to a numpy array and return
    def simulation(self):
        for server_idx, server in enumerate(self.appointments):
            for dia_idx, dia in enumerate(server):
                for slot_idx, slot in enumerate(dia):
                        #print(f'slot inicial{slot}')
                        
                        if slot.count(None)>0:
                                # si no hay paciente, idle system
                                self.idle_time_server[server_idx] += self.slot_time
                                continue
                        
                        # Only one patient assigned to the slot
                        if self.not_null(slot)==1:
                            # Assiged patient MISSES APPOINTMENT
                            if self.patients_list[slot[0]].attendance==False:
                                self.idle_time_server[server_idx]+=self.slot_time
                                self.no_shows+=1
                            # Assigned patient shows up
                            else:
                                if patient.protected == False:
                                    self.cwt_non_protected += self.compute_waiting_time(dia_idx, slot_idx, slot[0])
                                else:
                                    self.cwt_protected += self.compute_waiting_time(dia_idx, slot_idx, slot[0])

                                
                                self.service_time[server_idx]+=self.slot_time
                        
                        # There are no patients assigned to the slot
                        elif self.not_null(slot)==0:
                            if slot.count(None)>0:
                                self.idle_time_server[server_idx]+=self.slot_time
                                continue
                            self.idle_time_server[server_idx]+=self.slot_time
                        
                        # >1 patients assigned to the slot (OVERBOOKING)
                        else:
                            ids=[]
                            #print(len(slot))
                            for i in range(len(slot)):
                                if self.patients_list[slot[i]].attendance==True:
                                    ids.append(slot[i])
                                else:
                                    self.no_shows+=1
                            self.appointments[server_idx][dia_idx][slot_idx]=ids
                            
                            # More than one patient SHOW UP
                            if len(ids)>1:
                                
                                # 1.1 Is this the last slot schedule for the day- Si tengo que hacer overtime
                                if slot_idx==(len(self.appointments[server_idx][dia_idx])-1):
                                    self.over_time[server_idx]+=self.slot_time*(len(ids)-1)
                                
                                # 1.2 More than one patient SHOW-UP and is not the last slot for the server
                                else:
                                    temp_reasign=slot[1:]
                                    reasign=[]
                                    for i in range(len(temp_reasign)):
                                        if self.patients_list[temp_reasign[i]].attendance==True:
                                             reasign.append(temp_reasign[i])

                                    test = [None]
                                    test[0] = slot[0]
                                    reasign.extend(self.appointments[server_idx][dia_idx][slot_idx+1])
                                    
                                    #Como es un array natural d python no permite la indexacion de tipo [1,2,3]
                                    self.appointments[server_idx][dia_idx][slot_idx]= test
                                    self.appointments[server_idx][dia_idx][slot_idx+1]=reasign
                                    reasign=[]
                                if patient.protected == False:
                                    self.cwt_non_protected += self.compute_waiting_time(dia_idx, slot_idx, slot[0])
                                else:
                                    self.cwt_protected += self.compute_waiting_time(dia_idx, slot_idx, slot[0])
                            # Ninguno de los overbooked shows up
                            elif not ids:
                                self.idle_time_server[server_idx]+=self.slot_time
                            
                            # Solo uno de los overbooked shows up
                            else:
                                if patient.protected == False:
                                    self.cwt_non_protected += self.compute_waiting_time(dia_idx, slot_idx, slot[0])
                                else:
                                    self.cwt_protected += self.compute_waiting_time(dia_idx, slot_idx, slot[0])
    
                                self.service_time[server_idx]+=self.slot_time

    def not_null(self, lista):
        return max(len(lista) - lista.count(None), 0)
    
    def get_measures(self):
        measures = {
            "idle_time_server": self.idle_time_server.tolist(),  # Convert NumPy array to list
            "over_time": self.over_time.tolist(), # Convert NumPy array to list
            "no_shows": self.no_shows,
            "clients_total_waiting_time non protected class": self.cwt_non_protected,
            "clients_total_waiting_time protected class": self.cwt_protected,
            "service_time": self.service_time.tolist()  # Convert NumPy array to list
        }
        return measures

# Scheduling policy
### Create the routines to make the scheduling appointments
1. FIFO scheduling policy
2. Overbooking policy: Make overbooking with all high probabilities when there are no available slots for the day.
3. Make overbooking with expected probabilities:
    P(Both show up) = P(1-P(A)) * (1 -P(B))
4. Make overbooking 2ATBEG 

### Custom FIFO

In [70]:
time_per_slot = 20
work_hours = 10

slots_per_day = (60//time_per_slot) * work_hours
available_slots = slots_per_day * 7
extra_pct = 0.15
# np ceil rounds up
sample_size = int(np.ceil(available_slots + (available_slots * extra_pct)))

random_sample = random_patient_sample(patients, sample_size)

assert len(random_sample) == sample_size

In [71]:
# tratando de traducirla al cambio de paradigma
def custom_fifo(patient_sample, appointments, verbose=False):
    
    for patient in patient_sample:
        if not patient.assigned:
            for server in range(len(appointments)):

                desired_list_index = patient.day_of_call + patient.desired_lead_time

                # definiendo los dias, no puede ir mas alla del tamaño de dias por asignar
                start_day = patient.day_of_call
                end_day = min(patient.day_of_call + 6, len(appointments[server]))

                # iterando sobre los dias definidos
                for day in range(start_day, end_day):
                    if len(appointments[server][day]) < slots_per_day:
                        appointments[server][day].append(patient)
                        patient.assigned = True
                        if verbose:
                            print(f"--- P{patient.id} | Desired {day} | Assigned {day}")
                        break
                if patient.assigned:
                    break
                
            if not patient.assigned:
                for server in range(len(appointments)):
                    for day in range(len(appointments[server])):
                        if len(appointments[server][day]) < slots_per_day:
                            appointments[server][day].append(patient)
                            patient.assigned = True
                            if verbose:
                                print(f"--- P{patient.id} | Desired {day} | Assigned {day}")
                            break
                    if patient.assigned:
                        break
                if not patient.assigned:
                    if verbose:
                        print(f"* * * Full Schedule, can not assign more patients. Patient: {patient.id}")

    if verbose:
        print("Current Schedule:")
        for server, server_apps in enumerate(appointments):
            for day, day_apps in enumerate(server_apps):
                print(f"Server {server}, Day {day} ({len(day_apps)} Patients): {[patient.id for patient in day_apps]}")
        print("\n")

    refused_patients = len([patient for patient in patient_sample if not patient.assigned])

    return appointments, refused_patients

In [72]:
# contar cuantas listas no vacias hay en determinada lista
def count_non_empty_lists(list):
    count = 0 
    # que revise los que no estan vacios o no son none 
    for entry in list:
        if entry and entry[0] != None:
            count += 1
    return count

In [73]:
list_testing = [[None], [None], [], [1], [2], [3]]
count_non_empty_lists(list_testing)

3

In [74]:
# tratando de traducirla al cambio de paradigma
def custom_fifo(patient_sample, appointments, verbose=False):
    
    for patient in patient_sample:
        if not patient.assigned:
            for server in range(len(appointments)):

                # el paciente la desea en dia de la llamada + el tiempo deseado
                desired_list_index = patient.day_of_call + patient.desired_lead_time

                # definiendo los dias, no puede ir mas alla del tamaño de dias por asignar
                start_day = patient.day_of_call
                end_day = min(patient.day_of_call + 6, len(appointments[server]))

                # si el desired list index es mayor a 6, ese paciente no se asigna esta semana
                if desired_list_index > len(appointments[server]):
                    if verbose:
                        print(f"--- P{patient.id} | Desired {desired_list_index} | Not assigned this week")
                    break

                # si desea para esa semana y ese dia hay espacio
                if desired_list_index < len(appointments[server]) and count_non_empty_lists(appointments[server][desired_list_index]) < slots_per_day:

                    # asignarlo en el primer slot disponible de ese dia
                    for slot in range(len(appointments[server][desired_list_index])):
                        if not appointments[server][desired_list_index][slot]:
                            appointments[server][desired_list_index][slot] = patient
                            patient.assigned = True
                            if verbose:
                                print(f"--- P{patient.id} | Desired {desired_list_index} | Assigned {desired_list_index}")
                            break
                
                # si aun no se ha asignado, mirar en los dias siguientes al desired list index (hasta 6 dias) el primero donde se pueda asignar
                if not patient.assigned:
                    for day in range(desired_list_index, end_day):
                        # asignarlo en el primer slot disponible
                        for slot in range(len(appointments[server][day])):
                            if not appointments[server][day][slot]:
                                appointments[server][day][slot] = patient
                                patient.assigned = True
                                if verbose:
                                    print(f"--- P{patient.id} | Desired {desired_list_index} | Assigned {day}")
                                break
                    if patient.assigned:
                        break

                # iterando sobre los dias definidos
                for day in range(start_day, end_day):
                    if len(appointments[server][day]) < slots_per_day:
                        appointments[server][day].append(patient)
                        patient.assigned = True
                        if verbose:
                            print(f"--- P{patient.id} | Desired {day} | Assigned {day}")
                        break
                if patient.assigned:
                    break
                
            if not patient.assigned:
                for server in range(len(appointments)):
                    for day in range(len(appointments[server])):
                        if len(appointments[server][day]) < slots_per_day:
                            appointments[server][day].append(patient)
                            patient.assigned = True
                            if verbose:
                                print(f"--- P{patient.id} | Desired {day} | Assigned {day}")
                            break
                    if patient.assigned:
                        break
                if not patient.assigned:
                    if verbose:
                        print(f"* * * Full Schedule, can not assign more patients. Patient: {patient.id}")

    if verbose:
        print("Current Schedule:")
        for server, server_apps in enumerate(appointments):
            for day, day_apps in enumerate(server_apps):
                print(f"Server {server}, Day {day} ({len(day_apps)} Patients): {[patient.id for patient in day_apps]}")
        print("\n")

    refused_patients = len([patient for patient in patient_sample if not patient.assigned])

    return appointments, refused_patients

In [75]:
# tratando de traducirla al cambio de paradigma
def custom_fifo(patient_sample, appointments, verbose=False):
    
    non_assigned = 0
    for patient in patient_sample:
        for server in range(len(appointments)):

            patient.desired_lead_time = max(0, patient.desired_lead_time)

            if verbose:
                print(f"Eval P{patient.id} S{server} | Desired {patient.day_of_call + patient.desired_lead_time}")

            # el paciente desea en dia de la llamada + el tiempo deseado
            desired_list_index = patient.day_of_call + patient.desired_lead_time
            # definiendo los dias, no puede ir mas alla del tamaño de dias por asignar
            start_day = patient.day_of_call
            end_day = 6
            # end_day = min(patient.day_of_call + 6, len(appointments[server]))
            
            # si aun no se ha asignado, mirar en los dias siguientes al desired list index (hasta 6 dias) el primero donde se pueda asignar
            if not patient.assigned and desired_list_index < end_day:
                if verbose:
                    print(f"--- Trying to assign P{patient.id} | Desired {desired_list_index}")
                
                for day in range(desired_list_index, end_day+1):
                    # asignarlo en el primer slot disponible
                    for slot in range(len(appointments[server][day])):
                        if verbose:
                            # print(f"Va a evaluar en appointments[{server}][{day}][{slot}] {appointments[server][day][slot]}")
                            print("", end="")
                        
                        if appointments[server][day][slot][0] is None:
                            appointments[server][day][slot] = [patient]
                            patient.assigned = True

                            if verbose:
                                print(f"--- P{patient.id} | Desired {desired_list_index} | Assigned {day} : {patient.assigned}")
                                
                            break
                        if patient.assigned:
                            break
                    if patient.assigned:
                        break
                if patient.assigned:
                        break

                if not patient.assigned:
                    # ya este no se puede asignar
                    non_assigned += 1
                    if verbose:
                        print(f"--- P{patient.id} | Desired {desired_list_index} | Could not be assigned (Full Schedule)")
                    break

            elif desired_list_index > end_day:
                non_assigned += 1
                if verbose:
                    print(f"--- P{patient.id} | Desired {desired_list_index} | Wanted for next week")
                break

        if verbose:
            print("Current Schedule:")
            for server, server_apps in enumerate(appointments):
                for day, day_apps in enumerate(server_apps):
                    # print(f"Day {day} apps: {[pat_iter[0].id for pat_iter in day_apps if pat_iter[0].id is not None else None]}")
                    print(f"Day {day} apps: {[pat_iter[0].id if pat_iter[0] is not None else None for pat_iter in day_apps]}")
                    # print(f"Server {server}, Day {day} ({len(day_apps)} Patients): {[patient.id for patient in day_apps]}")
            print("\n")

    # refused_patients = len([patient for patient in patient_sample if not patient.assigned])

    return appointments, non_assigned

In [76]:
seed = 42
num_serves = 1
work_hours = 10
slot_time = 20
overbooking = 1.0
simulation_days = 7
num_slots_byday = (60//slot_time) * work_hours
num_simulations = 10 # Number of Monte Carlo simulations

available_slots = num_slots_byday* simulation_days 
extra_pct = 1.9
# np ceil rounds up
sample_size = int(np.ceil(available_slots + (available_slots * extra_pct)))

# Initialize the appointments
appointments = []
for _ in range(1):
    server = [] 
    for _ in range(7):
        dia = []
        for _ in range(num_slots_byday):
            slot = [None]
            dia.append(slot) 
        server.append(dia)
    appointments.append(server)

patient_sample = random_patient_sample(patients, 2050)

# for patient in patient_sample:
#     print(f"Patient {patient.id} - Desired {patient.day_of_call + patient.desired_lead_time}")

appointments_custom_fifo, non_assigned_custom_fifo = custom_fifo(patient_sample, appointments, verbose=0)

print(f"No assigned patients: {non_assigned_custom_fifo} : {non_assigned_custom_fifo/len(patient_sample)*100:.2f}%")
print(f"Appointments using custom FIFO: {appointments_custom_fifo}")

No assigned patients: 1840 : 89.76%
Appointments using custom FIFO: [[[[<__main__.Patient object at 0x000001A6E2906660>], [<__main__.Patient object at 0x000001A6E2A33C50>], [<__main__.Patient object at 0x000001A6E2F470E0>], [<__main__.Patient object at 0x000001A6E2D41D30>], [<__main__.Patient object at 0x000001A6F8962AB0>], [<__main__.Patient object at 0x000001A6DE651F10>], [<__main__.Patient object at 0x000001A6F899EAB0>], [<__main__.Patient object at 0x000001A6E2CBE990>], [<__main__.Patient object at 0x000001A6F88707A0>], [<__main__.Patient object at 0x000001A6E2D8ADE0>], [<__main__.Patient object at 0x000001A6FACE57C0>], [<__main__.Patient object at 0x000001A6E2D94D70>], [<__main__.Patient object at 0x000001A6F88AE750>], [<__main__.Patient object at 0x000001A6E2AD30E0>], [<__main__.Patient object at 0x000001A6E2F52000>], [<__main__.Patient object at 0x000001A6E2A72750>], [<__main__.Patient object at 0x000001A6E2F3B890>], [<__main__.Patient object at 0x000001A6F9AFB980>], [<__main__.

In [77]:
# patient_sample = random_patient_sample(patients, 2050)

In [78]:
appointments_custom_fifo, non_assigned_custom_fifo = custom_fifo(patient_sample, appointments, verbose=1)

Eval P16688 S0 | Desired 0
Current Schedule:
Day 0 apps: [16688, 25305, 63122, 48399, 74309, 6109, 76009, 44725, 67322, 50528, 102430, 50695, 69191, 30824, 63372, 27713, 62823, 87308, 102431, 56462, 68472, 27605, 55804, 52717, 62965, 16806, 54748, 5127, 69514, 50699]
Day 1 apps: [39790, 48377, 47615, 37507, 5682, 54928, 54781, 27655, 37084, 65657, 92185, 48885, 68902, 8641, 96361, 53375, 76243, 50537, 69072, 63180, 48147, 60103, 8629, 67640, 54886, 19644, 54873, 52080, 59060, 87279]
Day 2 apps: [5641, 96287, 96482, 96571, 48886, 68490, 96599, 27670, 5341, 96167, 53572, 52075, 63060, 56680, 41140, 27617, 60205, 2248, 9381, 19417, 63194, 61478, 102129, 56380, 67364, 48694, 76242, 61663, 67186, 92091]
Day 3 apps: [19558, 91918, 58616, 72069, 73800, 67638, 54497, 8740, 1936, 67427, 68898, 16675, 44981, 69324, 63772, 44544, 63087, 87286, 19367, 56359, 57500, 51432, 53477, 6073, 63583, 57625, 58612, 19705, 63319, 44873]
Day 4 apps: [67188, 25178, 102222, 13679, 51365, 8559, 6245, 19577, 6152

--- P52701 | Desired 0 | Could not be assigned (Full Schedule)
Current Schedule:
Day 0 apps: [16688, 25305, 63122, 48399, 74309, 6109, 76009, 44725, 67322, 50528, 102430, 50695, 69191, 30824, 63372, 27713, 62823, 87308, 102431, 56462, 68472, 27605, 55804, 52717, 62965, 16806, 54748, 5127, 69514, 50699]
Day 1 apps: [39790, 48377, 47615, 37507, 5682, 54928, 54781, 27655, 37084, 65657, 92185, 48885, 68902, 8641, 96361, 53375, 76243, 50537, 69072, 63180, 48147, 60103, 8629, 67640, 54886, 19644, 54873, 52080, 59060, 87279]
Day 2 apps: [5641, 96287, 96482, 96571, 48886, 68490, 96599, 27670, 5341, 96167, 53572, 52075, 63060, 56680, 41140, 27617, 60205, 2248, 9381, 19417, 63194, 61478, 102129, 56380, 67364, 48694, 76242, 61663, 67186, 92091]
Day 3 apps: [19558, 91918, 58616, 72069, 73800, 67638, 54497, 8740, 1936, 67427, 68898, 16675, 44981, 69324, 63772, 44544, 63087, 87286, 19367, 56359, 57500, 51432, 53477, 6073, 63583, 57625, 58612, 19705, 63319, 44873]
Day 4 apps: [67188, 25178, 102222, 1

In [79]:
print(f"No assigned patients: {non_assigned_custom_fifo} : {non_assigned_custom_fifo/len(patient_sample)*100:.2f}%")
print(f"Appointments using custom FIFO: {appointments_custom_fifo}")

No assigned patients: 1840 : 89.76%
Appointments using custom FIFO: [[[[<__main__.Patient object at 0x000001A6E2906660>], [<__main__.Patient object at 0x000001A6E2A33C50>], [<__main__.Patient object at 0x000001A6E2F470E0>], [<__main__.Patient object at 0x000001A6E2D41D30>], [<__main__.Patient object at 0x000001A6F8962AB0>], [<__main__.Patient object at 0x000001A6DE651F10>], [<__main__.Patient object at 0x000001A6F899EAB0>], [<__main__.Patient object at 0x000001A6E2CBE990>], [<__main__.Patient object at 0x000001A6F88707A0>], [<__main__.Patient object at 0x000001A6E2D8ADE0>], [<__main__.Patient object at 0x000001A6FACE57C0>], [<__main__.Patient object at 0x000001A6E2D94D70>], [<__main__.Patient object at 0x000001A6F88AE750>], [<__main__.Patient object at 0x000001A6E2AD30E0>], [<__main__.Patient object at 0x000001A6E2F52000>], [<__main__.Patient object at 0x000001A6E2A72750>], [<__main__.Patient object at 0x000001A6E2F3B890>], [<__main__.Patient object at 0x000001A6F9AFB980>], [<__main__.

### FIFO

In [80]:
def fifo(patient, appointments):
    if not patient.assigned:
        for server in range(len(appointments)):
            start_day = patient.day_of_call
            end_day = min(patient.day_of_call + 6, len(appointments[server]))
            for dia in range(start_day, end_day):
                for slot in range(len(appointments[server][dia])):
                    for id in range(len(appointments[server][dia][slot])):
                        if appointments[server][dia][slot][id] is None:
                            appointments[server][dia][slot][id] = patient.id
                            patient.num_slot=slot
                            patient.assigned = True
                                #print(f"Patient {patient.id} assigned to the first available slot within the next {simulation_days} days")                            
                            break  # Break from innermost loop
                    if patient.assigned:
                            break  # Break from middle loop
                if patient.assigned:
                        break  # Break from outer loop
            if patient.assigned:
                    break  # Break from outer loop
    return appointments

### Overbooking
Establecemos segun un threshold que pacientes se les puede hacer overbooking
1. Si se les puede hacer overbooking se busca un slot con un paciente ya asignado y se le asigna como segundo paciente.
2. Si no se le puede hacer overbooking, se busca el primer el slot que se encuentre vacio y se le asigna

In [81]:
def rule_Overbooking(patient, appointments, threshold, train_model):

    # Establece si un paciente si le puede hacer overbooking segun su probabilidad (ALTA)

    # Predice proba de inasistencia con el respectivo modelo
    patient.predict_proba(train_model)
    
    # Decide si overbook o no basado en la prediccion
    overbook=True if patient.proba>threshold else False
    
    # Si el paciente aun no se ha asignado 
    if not patient.assigned:

        start_day=patient.day_of_call
        
        # el dia final es el ultimo de la ventana de tiempo 
        # end_day=min(patient.day_of_call+6,len(appointments[0]))
        end_day=len(appointments[0])
        
        # Itera desde el dia que llama hasta el final
        for dia in range(start_day, end_day):

            # Itera sobre los slots del dia
            if not patient.assigned:
                for slot in range(len(appointments[0][dia])):
                    for server in range(len(appointments)):        
                        # Itera dentro de los slots para revisar asignaciones
                        for id in range(len(appointments[server][dia][slot])):
                            if overbook:
                                # Overbook si hay exactamente un paciente en ese slot
                                if appointments[server][dia][slot][id] is not None and len(appointments[server][dia][slot])==1:
                                    appointments[server][dia][slot].append(patient.id)
                                    patient.num_slot = slot
                                    patient.assigned = True
                                    break
                            if patient.assigned:
                                break  # romper ciclos si ya se asigno
                        if patient.assigned:
                            break  
                    if patient.assigned:
                        break 
                if patient.assigned:
                    break 

            # Si aun no se asigna es porque no se debe hacer overbooking
            if not patient.assigned:
                for slot in range(len(appointments[0][dia])):
                    for server in range(len(appointments)):        
                        for id in range(len(appointments[server][dia][slot])):
                            # asigna en un slot vacio (no overbooking)
                            if appointments[server][dia][slot][id] is  None :
                                appointments[server][dia][slot][id]= patient.id
                                patient.num_slot=slot
                                patient.assigned=True
                                break
                            if patient.assigned:
                                break
                        if patient.assigned:
                            break
                    if patient.assigned:
                        break
                if patient.assigned:
                    break  # Break from middle loop
            if patient.assigned:
                break
    
    return appointments

In [82]:
# Objetivo calcular la matriz de probabilidad solo una vez al principio y solo con los slot disponible 
# Retorna una matriz de probabilidad con todos los slots disponibles
# Retorna proba_global que es donde se encuentra disponibile y es la menor probabilidad
# Retorna proba_filled que es donde se encuentra ocupado y es la menor probabilidad
def get_proba_matrix(patient, appointments,model,threshold):
    proba_global={"proba":1000,"server":None,"dia":None,"slot":None}
    proba_filled={"proba":1000,"server":None,"dia":None,"slot":None}
    # Get the dimensions
    length1 = len(appointments)# serves
    length2 = len(appointments[0])#days
    length3 = len(appointments[0][0])#slots
    overbook=False
    cont_overbook=0

    # Create the NumPy array filled with zeros
    proba_matrix = np.ones((length1, length2, length3), dtype=float)
    dia_inicio=patient.day_of_call
    for server in range(len(appointments)):
        for dia in range(dia_inicio,len(appointments[server])):
            for slot in range(len(appointments[server][dia])):
                    if appointments[server][dia][slot][0] is None:
                        patient.real_lead_time=(dia+(slot/len(appointments[server][dia])))-patient.day_of_call 
                        proba_new=patient.predict_proba(model)
                        proba_matrix[server,dia,slot] = proba_new
                        if proba_new>threshold and dia < 3 :
                             cont_overbook+=1
                        if proba_new<proba_global["proba"]:
                            proba_global={"proba":proba_new,"server":server,"dia":dia,"slot":slot}
                    else:
                        patient.real_lead_time=(dia+(slot/len(appointments[server][dia])))-patient.day_of_call
                        proba_new=patient.predict_proba(model)
                        if proba_new>threshold and dia < 3 :
                             cont_overbook+=1
                        proba_matrix[server,dia,slot] = proba_new
                        if proba_new<proba_filled["proba"]:
                            proba_filled={"proba":proba_new,"server":server,"dia":dia,"slot":slot} 
    if cont_overbook>2*len(appointments[0])*len(appointments)*0.6:
         overbook=True
    return proba_matrix, proba_global, proba_filled,overbook

In [83]:
def low_probability(patient, appointments,threshold, general_list,train_model):
    print(type(appointments))
    proba_matrix, best_proba, filled_proba,over=get_proba_matrix(patient,appointments,train_model,0.4)
    
    
    # 1. Reviso si el lugar con menor probabilidad esta vacio para ubicarlo en ese lugar
    if best_proba["server"] is None and best_proba["proba"]<threshold:
        appointments[best_proba["server"]][best_proba["dia"]][best_proba["slot"]][0]=patient.id
        patient.assigned=True
    # 2. Si no fue asignado, 
    if not patient.assigned:
        start_day=patient.day_of_call
        #end_day=min(patient.day_of_call+6,len(appointments[0]))
        end_day=len(appointments[0])
        for dia in range(start_day,end_day):
            if not patient.assigned:
                for server in range(len(appointments)):
                    #Revisa el primer dia para no afectar su leadtime indirecto en el sistema
                    if over:
                        for slot in range(len(appointments[0][dia])):
                            for server in range(len(appointments)):        
                                for id in range(len(appointments[server][dia][slot])):
                                    if appointments[server][dia][slot][id] is not None and len(appointments[server][dia][slot])==1:
                                    # Asignar a dos pacientes con alta probabilidad
                                        if general_list[appointments[server][dia][slot][0]].proba>threshold and proba_matrix[server,dia,slot]>threshold:
                                            patient.proba=proba_matrix[server,dia,slot]
                                            appointments[server][dia][slot].append(patient.id)
                                            patient.num_slot=slot
                                            patient.assigned=True
                                            break
                                        else :
                                            appointments[server][dia][slot][id]= patient.id
                                            patient.num_slot=slot
                                            patient.assigned=True
                                            break
                                    if patient.assigned:
                                        break
                                if patient.assigned:
                                    break
                            if patient.assigned:
                                break
                else:
                    for slot in range(len(appointments[0][dia])):
                        for server in range(len(appointments)):        
                            for id in range(len(appointments[server][dia][slot])):
                                if appointments[server][dia][slot][id] is  None :
                                    appointments[server][dia][slot][id]= patient.id
                                    patient.num_slot=slot
                                    patient.assigned=True
                                    break
                                if patient.assigned:
                                    break
                            if patient.assigned:
                                break
                        if patient.assigned:
                            break
                    if patient.assigned:
                        break
                if patient.assigned:
                    break
            else:
                break
    return appointments

### Call a rule over list of patients

In [84]:
def call_a_rule (patient_list, appointments, name_rule, ml_model):
    refused_patients=0
    
    # Determinar procedimiento a usar segun rule name
    for patient in patient_list:
        if name_rule=='fifo':
            appointments = fifo(patient,appointments)

        elif name_rule == 'custom_fifo':
            
            # # Initialize the appointments
            # appointment_entry_list = []
            # for _ in range(1):
            #     server = [] 
            #     for _ in range(7):
            #         dia = []
            #         for _ in range(num_slots_byday):
            #             slot = [None]
            #             dia.append(slot) 
            #         server.append(dia)
            #     appointment_entry_list.append(server)

            appointments, refused_patients = custom_fifo(patient_list, appointments)
        
        elif name_rule=='overbooking_simple':

            # threshold og era 0.6
            appointments = rule_Overbooking(patient, appointments, 0.000001, ml_model)

        elif name_rule=='low_probability':
            appointments = low_probability(patient, appointments, 0.6, patient_list, ml_model)
        else:
            print("Unknown name_rule")
    
    if not patient.assigned:
                    #print(f"Patient {patient.id} could not be assigned within the next {simulation_days} days")
        refused_patients += 1
    
    return appointments, refused_patients

## Call the simulation and changing the patient attendance

In [85]:
# OPCION CON PROBA
# Run the simulation, with already defined schedule (appointments list)

def scheduling_simulation(patients_data, appointments, simulation_days, 
                          num_serves, num_hours_byday, slot_time, 
                          overbooking,ml_model, benchmark=None):    
    
    # Determine attendance for all patients comparing random P with P(no show)
    for patient in patients_data:

        if benchmark is None:
            patient.attendance=False if np.random.uniform(0, 1) < patient.predict_proba(ml_model) else True
        else:
            patient.attendance=False if np.random.uniform(0, 1) < benchmark else True
        #patient.print_non_boolean_attributes()

    clinic = Clinic(patients_data, appointments, simulation_days, num_serves, simulation_days, num_hours_byday, slot_time, overbooking)
    clinic.simulation()
    
    # Return measures
    return clinic.get_measures()

In [86]:
# OPCION EVALUANDO CON PPV, NPV del modelo
# Run the simulation, with already defined schedule (appointments list)

def scheduling_simulation(patients_data, appointments, simulation_days, 
                          num_serves, num_hours_byday, slot_time, 
                          overbooking,ml_model, benchmark=None):    
    
    protected_ppv = 0.2
    non_protected_ppv = 0.6

    protected_npv = 0.9
    protected_npv = 0.91

    # Determine attendance for all patients comparing random P with P(no show)
    for patient in patients_data:

        # defining comparison probability taking into account model precisions
        if patient.protected:
            comparison_proba = protected_ppv
        else:
            comparison_proba = non_protected_ppv

        if benchmark is None:
            patient.attendance=False if np.random.uniform(0, 1) < comparison_proba else True
        else:
            patient.attendance=False if np.random.uniform(0, 1) < benchmark else True
        #patient.print_non_boolean_attributes()

    clinic = Clinic(patients_data, appointments, simulation_days, num_serves, simulation_days, num_hours_byday, slot_time, overbooking)
    clinic.simulation()
    
    # Return measures
    return clinic.get_measures()

### Computing the confidence intervals and margin errors

In [87]:
from scipy import stats
def get_margin_errors(data_list, confidence=0.95):
    # Extract a list of measure names (assuming consistent keys across dictionaries)
    measure_names = list(data_list[0].keys())

    # Check if all dictionaries have the same set of measures
    for data_dict in data_list:
        if set(data_dict.keys()) != set(measure_names):
            raise ValueError("Dictionaries in the list must have the same set of measures.")
    margin_errors = []
    for measure_name in measure_names:
        # Check if measure is a list
        is_list_measure = isinstance(data_list[0][measure_name], list)

        if is_list_measure:
            # Combine all values into a single list
            measure_values = [val for data_dict in data_list for val in data_dict[measure_name]]
        else:
            # Extract measure values from each dictionary (single value case)
            measure_values = [data_dict[measure_name] for data_dict in data_list]

        # Calculate the margin of error
        _, margin_of_error = confidence_interval(measure_values, confidence=confidence)

        # Append margin of error to the list
        margin_errors.append(margin_of_error)

    return margin_errors

def check_convergence(history, diff=0.01):
    if len(history) < 2:
        return False  # Convergence cannot be checked with less than two lists in history
    
    last_margin_errors = history[-1]
    second_last_margin_errors = history[-2]
    
    # Check if the margin errors for all measures have converged
    for last_margin, second_last_margin in zip(last_margin_errors, second_last_margin_errors):
        scale = max(abs(last_margin), abs(second_last_margin))
        if abs(last_margin - second_last_margin) > diff * scale:
            return False  # Measures haven't converged
    
    return True  # All measures have converged within the threshold

def confidence_interval(data, confidence=0.95):
  """ This function calculates the confidence interval for a given set of data.
  Args: data: A list of numerical values (or a single numerical value wrapped in a list).
        confidence: The desired confidence level (default: 0.95).

  Returns: A tuple containing the mean and the margin of error.
  """
  if len(data) == 1:
    data = data[0]  # Extract the single value if only one element

  n = len(data)
  m = np.mean(data)
  std_err = np.std(data, ddof=1) / np.sqrt(n)
  t_value = stats.t.ppf((1 + confidence) / 2, n - 1)
  margin_of_error = t_value * std_err
  return m, margin_of_error


# Define simulation parameters


In [88]:
seed = 42
num_serves = 1
work_hours = 10
slot_time = 20
overbooking = 1.0
simulation_days = 7
num_slots_byday = (60//slot_time) * work_hours
num_simulations = 10 # Number of Monte Carlo simulations

available_slots = num_slots_byday* simulation_days 

# OG era 1.9
extra_pct = 15
# np ceil rounds up
sample_size = int(np.ceil(available_slots + (available_slots * extra_pct)))

In [89]:
# Initialize the appointments
appointments = []
for _ in range(num_serves):
    server = [] 
    for _ in range(simulation_days):
        dia = []
        for _ in range(num_slots_byday):
            slot = [None]
            dia.append(slot) 
        server.append(dia)
    appointments.append(server)
# Create patient data

In [90]:
# Apply scheduling rule
test_patient_list=random_patient_sample(patients,sample_size)
test_patient_list = asignar_dia(test_patient_list,simulation_days)

appointments, num_refused = call_a_rule (test_patient_list, appointments, "overbooking_simple" ,rf)
print(f"Number of refused patients: {num_refused}")

Number of refused patients: 1


In [91]:
# Initialize the appointments
appointments = []
for _ in range(num_serves):
    server = [] 
    for _ in range(simulation_days):
        dia = []
        for _ in range(num_slots_byday):
            slot = [None]
            dia.append(slot) 
        server.append(dia)
    appointments.append(server)
# Create patient data

# Apply scheduling rule
test_patient_list=random_patient_sample(patients,sample_size)
test_patient_list = asignar_dia(test_patient_list,simulation_days)

appointments, num_refused = call_a_rule (test_patient_list, appointments, "overbooking_simple" ,rf)
print(f"Number of refused patients: {num_refused}")

print(f"Appointments: {appointments}")

Number of refused patients: 1
Appointments: [[[[0, 2], [1, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18], [19, 20], [22, 23], [24, 25], [26, 27], [28, 29], [30, 32], [33, 35], [36, 37], [38, 39], [40, 41], [43, 44], [46, 47], [49, 51], [50, 52], [53, 54], [55, 56], [57, 58], [59, 60], [61, 62], [63, 64], [65, 66]], [[67, 68], [69, 70], [71, 72], [73, 74], [75, 76], [77, 78], [79, 80], [81, 82], [83, 84], [85, 86], [87, 88], [89, 90], [91, 92], [93, 94], [95, 96], [97, 98], [99, 100], [101, 103], [104, 105], [106, 107], [108, 109], [110, 111], [113, 114], [115, 116], [117, 118], [120, 121], [122, 123], [124, 125], [126, 127], [128, 129]], [[130, 131], [132, 134], [133, 135], [136, 138], [137, 139], [140, 142], [143, 144], [145, 147], [146, 149], [148, 151], [152, 153], [154, 155], [156, 157], [158, 159], [160, 161], [162, 163], [164, 165], [166, 167], [168, 169], [170, 171], [172, 173], [175, 176], [177, 178], [179, 180], [181, 182], [183, 184], [185, 186], [187, 1

In [92]:
appointments

[[[[0, 2],
   [1, 4],
   [5, 6],
   [7, 8],
   [9, 10],
   [11, 12],
   [13, 14],
   [15, 16],
   [17, 18],
   [19, 20],
   [22, 23],
   [24, 25],
   [26, 27],
   [28, 29],
   [30, 32],
   [33, 35],
   [36, 37],
   [38, 39],
   [40, 41],
   [43, 44],
   [46, 47],
   [49, 51],
   [50, 52],
   [53, 54],
   [55, 56],
   [57, 58],
   [59, 60],
   [61, 62],
   [63, 64],
   [65, 66]],
  [[67, 68],
   [69, 70],
   [71, 72],
   [73, 74],
   [75, 76],
   [77, 78],
   [79, 80],
   [81, 82],
   [83, 84],
   [85, 86],
   [87, 88],
   [89, 90],
   [91, 92],
   [93, 94],
   [95, 96],
   [97, 98],
   [99, 100],
   [101, 103],
   [104, 105],
   [106, 107],
   [108, 109],
   [110, 111],
   [113, 114],
   [115, 116],
   [117, 118],
   [120, 121],
   [122, 123],
   [124, 125],
   [126, 127],
   [128, 129]],
  [[130, 131],
   [132, 134],
   [133, 135],
   [136, 138],
   [137, 139],
   [140, 142],
   [143, 144],
   [145, 147],
   [146, 149],
   [148, 151],
   [152, 153],
   [154, 155],
   [156, 157],
   [1

In [93]:
patient_test=patients[3195]
ml_model = rf
proba_matrix, best_proba, filled_proba, over = get_proba_matrix(patient_test, appointments, ml_model,threshold=0.4)
print(np.around(proba_matrix, decimals=4))

[[[0.17   0.17   0.17   0.17   0.17   0.17   0.17   0.17   0.17   0.17
   0.17   0.17   0.17   0.17   0.17   0.17   0.25   0.25   0.25   0.25
   0.25   0.25   0.25   0.25   0.25   0.25   0.25   0.25   0.25   0.25  ]
  [0.25   0.25   0.25   0.25   0.25   0.25   0.25   0.25   0.25   0.25
   0.25   0.25   0.25   0.25   0.25   0.25   0.24   0.24   0.24   0.24
   0.24   0.24   0.24   0.24   0.24   0.24   0.24   0.24   0.24   0.24  ]
  [0.24   0.24   0.24   0.24   0.24   0.24   0.24   0.24   0.24   0.24
   0.24   0.24   0.24   0.24   0.24   0.24   0.32   0.32   0.32   0.32
   0.32   0.32   0.32   0.32   0.32   0.32   0.32   0.32   0.32   0.32  ]
  [0.32   0.33   0.33   0.33   0.33   0.33   0.33   0.33   0.33   0.33
   0.33   0.33   0.33   0.33   0.33   0.33   0.29   0.29   0.29   0.29
   0.29   0.29   0.29   0.29   0.29   0.29   0.29   0.29   0.29   0.29  ]
  [0.29   0.29   0.29   0.29   0.29   0.29   0.29   0.29   0.29   0.29
   0.29   0.29   0.29   0.29   0.29   0.29   0.36   0.36   0.36  

In [94]:
best_proba

{'proba': 1000, 'server': None, 'dia': None, 'slot': None}

In [95]:
filled_proba

{'proba': 0.17, 'server': 0, 'dia': 0, 'slot': 0}

# MonteCarlo

In [96]:
# Initialize lists to store results
all_measures = []
historic_margin_errors = []
converge = False
iterations = 0

num_iterations = 5

# Perform Monte Carlo simulation
while not converge and iterations < num_iterations:

    iterations += 1

    # se hace el muestreo de los pacientes
    random_patient_list = random_patient_sample(patients,sample_size)
    # se asigna el dia en el que llaman
    random_patient_list = asignar_dia(random_patient_list,simulation_days)
    appointments, num_refused = call_a_rule(random_patient_list, appointments, "overbooking_simple",rf)

    # measures= scheduling_simulation(patients_data, appointments, simulation_days, num_serves, num_hours_byday, slot_time, overbooking, benchmark_umbral=0.6)
    
    measures = scheduling_simulation(random_patient_list, appointments, simulation_days, num_serves, work_hours, slot_time, overbooking,rf)
    all_measures.append(measures)
    #print("measures:", measures)  # Debug print
    if iterations > 10 : 
        historic_margin_errors.append(get_margin_errors(all_measures)) # Debug
        converge = check_convergence(historic_margin_errors)
    
    print(f"Iter {iterations}/{num_iterations} ran successfully") # Debug print

print(f"All Iterations Ran Successfully")# Debug print
# Convert measures to DataFrame for analysis
measures_df = pd.DataFrame(all_measures)

Iter 1/5 ran successfully
Iter 2/5 ran successfully
Iter 3/5 ran successfully
Iter 4/5 ran successfully
Iter 5/5 ran successfully
All Iterations Ran Successfully


In [97]:
# Initialize lists to store results
all_measures = []
historic_margin_errors = []
converge = False
iterations = 0

num_iterations = 5

# Perform Monte Carlo simulation
while not converge and iterations < num_iterations:

    iterations += 1

    # se hace el muestreo de los pacientes
    random_patient_list = random_patient_sample(patients,sample_size)
    # se asigna el dia en el que llaman
    random_patient_list = asignar_dia(random_patient_list,simulation_days)

    appointments, num_refused = call_a_rule(random_patient_list, appointments, "fifo",rf)

    # measures= scheduling_simulation(patients_data, appointments, simulation_days, num_serves, num_hours_byday, slot_time, overbooking, benchmark_umbral=0.6)
    
    measures = scheduling_simulation(random_patient_list, appointments, simulation_days, num_serves, work_hours, slot_time, overbooking,rf)
    all_measures.append(measures)
    #print("measures:", measures)  # Debug print
    if iterations > 10 : 
        historic_margin_errors.append(get_margin_errors(all_measures)) # Debug
        converge = check_convergence(historic_margin_errors)
    print(f"Ran Successfully Iter {iterations}/{num_iterations}") # Debug print

print(f"Ran Successfully All Iterations")# Debug print
# Convert measures to DataFrame for analysis
measures_df = pd.DataFrame(all_measures)

Ran Successfully Iter 1/5
Ran Successfully Iter 2/5
Ran Successfully Iter 3/5
Ran Successfully Iter 4/5
Ran Successfully Iter 5/5
Ran Successfully All Iterations


In [98]:
# Initialize lists to store results
all_measures = []
historic_margin_errors = []
converge = False
iterations = 0

num_iterations = 10

# Perform Monte Carlo simulation
while not converge and iterations < num_iterations:

    iterations += 1

    # Initialize appointments (o luego hay bug en las asignaciones)
    appointments = []
    for _ in range(num_serves):
        server = [] 
        for _ in range(simulation_days):
            dia = []
            for _ in range(num_slots_byday):
                slot = [None]
                dia.append(slot) 
            server.append(dia)
        appointments.append(server)

    # se hace el muestreo de los pacientes
    random_patient_list = random_patient_sample(patients,sample_size)
    # se asigna el dia en el que llaman
    random_patient_list = asignar_dia(random_patient_list,simulation_days)

    appointments, num_refused = call_a_rule(random_patient_list, appointments, "overbooking_simple", rf)

    # measures= scheduling_simulation(patients_data, appointments, simulation_days, num_serves, num_hours_byday, slot_time, overbooking, benchmark_umbral=0.6)
    
    measures = scheduling_simulation(random_patient_list, appointments, simulation_days, num_serves, work_hours, slot_time, overbooking,rf)
    all_measures.append(measures)
    #print("measures:", measures)  # Debug print
    if iterations > 10 : 
        historic_margin_errors.append(get_margin_errors(all_measures)) # Debug
        converge = check_convergence(historic_margin_errors)
    print(f"Ran Successfully Iter {iterations}/{num_iterations}") # Debug print

print(f"Ran Successfully All Iterations")# Debug print
# Convert measures to DataFrame for analysis
measures_df = pd.DataFrame(all_measures)

Ran Successfully Iter 1/10
Ran Successfully Iter 2/10
Ran Successfully Iter 3/10
Ran Successfully Iter 4/10
Ran Successfully Iter 5/10
Ran Successfully Iter 6/10
Ran Successfully Iter 7/10
Ran Successfully Iter 8/10
Ran Successfully Iter 9/10
Ran Successfully Iter 10/10
Ran Successfully All Iterations


## Final measures 

In [99]:
from scipy.stats import t

# Calculate mean, confidence interval, maximum, and minimum for each column
summary = []
for col in measures_df.columns:
    if isinstance(measures_df[col].iloc[0], list):  # Check if the column contains lists
        values = np.concatenate(measures_df[col].values)
        mean, margin_of_error = confidence_interval(values)
        summary.append({
            "column": col,
            "mean": mean,
            "confidence_interval": (mean - margin_of_error, mean + margin_of_error),
            "maximum": np.max(values),
            "minimum": np.min(values)
        })
    else:
        mean, margin_of_error = confidence_interval(measures_df[col])
        summary.append({
            "column": col,
            "mean": mean,
            "confidence_interval": (mean - margin_of_error, mean + margin_of_error),
            "maximum": measures_df[col].max(),
            "minimum": measures_df[col].min()
        })

# Create summary DataFrame
summary_measures_df = pd.DataFrame(summary)

# Print summary DataFrame
print(summary_measures_df)

                                           column     mean  \
0                                idle_time_server      4.0   
1                                       over_time   2538.0   
2                                        no_shows     83.3   
3  clients_total_waiting_time non protected class  23618.0   
4      clients_total_waiting_time protected class      0.0   
5                                    service_time     76.0   

                         confidence_interval  maximum  minimum  
0  (-2.0324191006426435, 10.032419100642644)     20.0      0.0  
1   (2361.9388594967604, 2714.0611405032396)   2800.0   2140.0  
2     (74.65598071655543, 91.94401928344456)    103.0     70.0  
3     (22520.65400924322, 24715.34599075678)  25920.0  21220.0  
4                                 (0.0, 0.0)      0.0      0.0  
5   (50.053584374575614, 101.94641562542438)    120.0     20.0  


In [100]:
import numpy as np

# Supongamos que tienes una matriz proba_matrix
proba_matrix = np.ones((5, 4, 3), dtype=float)  # Ejemplo de matriz llena de unos
length1, length2, length3 = proba_matrix.shape  # Obtener las dimensiones

proba_matrix[0,0,2]=-0.3
# Encontrar el índice del valor mínimo en proba_matrix[0][0] a lo largo de la tercera dimensión
min_index = np.argmin(proba_matrix[0, 0])

# Convertir el índice plano en una tupla de índices multidimensionales
min_index_3d = np.unravel_index(min_index, proba_matrix[0, 0].shape)

print("Índice del valor mínimo en proba_matrix[0][0]:", min_index_3d[0])
print(type(min_index_3d))

Índice del valor mínimo en proba_matrix[0][0]: 2
<class 'tuple'>


### Pruebas 15 Abril 